In [252]:
import numpy as np
import pandas as pd

In [253]:
key = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'conversions')

def create_dict_key(dataframe):
    '''
    '''
    conversion_key = dict()
    for year in dataframe.columns[1:]:
        values = list(dataframe[year])
        categories = list(dataframe['categories'])
        temp = dict()
        for tup in tuple(zip(categories, values)):
            temp[tup[0]] = tup[1]
        
        temp['USD'] = 1
        conversion_key[year] = temp
            
    return conversion_key

conversion_key = create_dict_key(key)

In [250]:
df = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'raw data')

def adjustments(raw_data, conversion_key, fy_starts=13):
    '''
    Takes inputs of raw_data and conversion key and
    converts to real money terms.
    
    Inputs:
        raw_data: pandas dataframe of raw data Excel
        raw_data: pandas dataframe of raw data Excel
        
    Outputs:
        adjusted_data: pandas dataframe of adjusted data
    '''
    columns = raw_data.columns[fy_starts:]
    raw_data['Region'] = raw_data['Type']
    
    for index, row in raw_data.iterrows():
        currency = row['Currency']
        for year in columns:
            year_match = int(year[3:])
            infl = conversion_key[year_match]['Inflation Adjustment']
            curr = conversion_key[year_match][currency]
            row[year] = (float(row[year])/curr)*infl
        
        if 'US' in row['Region']:
            row['Region'] = 'US'
        elif 'Total' in row['Region']:
            row['Region'] = 'WW'
        else:
            row['Region'] = 'ExUS'
            
        raw_data.loc[index] = row
        
    clean = raw_data 
    
    return clean

clean = adjustments(df, conversion_key)

def summary(clean_data, fy_starts=13):
    '''
    Get from array 2 to array 3
    '''
    # First simulate the dataframr
    data = []
    names = list(clean_data['Proper Name'].unique())
    for name in names:
        data.append([name, 'US'])
        data.append([name, 'WW'])
        data.append([name, 'ExUS'])
    
    data = pd.DataFrame(data, columns = ['Proper Name', 'Region']) 
    
    # Add with original data to create empty rows
    years = clean_data.columns[fy_starts:]
    clean = clean_data.groupby(['Proper Name', 'Region'], as_index=False)[years].sum()
    
    clean = clean.merge(data, how='right', on=['Proper Name', 'Region'])
    
    # Place holder for summary helper
    
    return clean.fillna(0)


summary(clean)

,Proper Name,Region,FY-2007,FY-2008,FY-2009,FY-2010,FY-2011,FY-2012,FY-2013,FY-2014,...,FY-2029,FY-2030,FY-2031,FY-2032,FY-2033,FY-2034,FY-2035,FY-2036,FY-2037,FY-2038
0,Actemra,US,0.0,0.0,0.0,75.879615,208.489888,331.554468,431.294194,556.352391,...,335.725889,315.654785,335.012030,343.125495,362.802547,358.359781,363.288261,0.0,0.0,0.0
1,Actemra,WW,0.0,0.0,0.0,519.382885,913.806742,1158.377021,1424.369677,1677.279130,...,1026.982709,950.614498,983.526357,1026.718156,1067.725867,1061.260590,1244.462648,0.0,0.0,0.0
2,Actemra,ExUS,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,Adempas,US,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,388.825103,334.938968,300.437226,296.269494,296.007724,88.168056,61.717639,0.0,0.0,0.0
4,Adempas,WW,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,84.362632,...,238.420843,234.891855,204.027394,86.399891,71.240784,59.505770,26.347257,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Xtandi,WW,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.159675,0.650976,...,2.004867,1.371162,0.980480,0.760915,0.347654,0.278064,0.000000,0.0,0.0,0.0
278,Xtandi,ExUS,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
279,Yervoy,US,0.0,0.0,0.0,0.000000,425.068000,650.479600,730.814733,893.836300,...,608.303033,475.140206,287.319594,242.144549,159.085817,182.393814,145.915051,0.0,0.0,0.0
280,Yervoy,WW,0.0,0.0,0.0,0.000000,473.760000,912.999200,1226.304000,1648.995600,...,1052.208894,859.211704,529.341355,451.136010,362.668629,615.523927,600.701670,0.0,0.0,0.0


In [239]:
new_df = summary(clean)
new_df.fillna(0)

for index, row in new_df.iterrows():
    for year in list(new_df.columns[2:]):
        if (row['Region'] == 'US' and row)

SyntaxError: unexpected EOF while parsing (<ipython-input-239-10496c1bcbed>, line 6)

In [241]:
new_df = summary(clean)
new_df.fillna(0)


['FY-2007',
 'FY-2008',
 'FY-2009',
 'FY-2010',
 'FY-2011',
 'FY-2012',
 'FY-2013',
 'FY-2014',
 'FY-2015',
 'FY-2016',
 'FY-2017',
 'FY-2018',
 'FY-2019',
 'FY-2020',
 'FY-2021',
 'FY-2022',
 'FY-2023',
 'FY-2024',
 'FY-2025',
 'FY-2026',
 'FY-2027',
 'FY-2028',
 'FY-2029',
 'FY-2030',
 'FY-2031',
 'FY-2032',
 'FY-2033',
 'FY-2034',
 'FY-2035',
 'FY-2036',
 'FY-2037',
 'FY-2038']

In [245]:
df2 = pd.read_excel('../data/new data/new data.xlsx', sheet_name = 'new data')
df2

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Manufacturer,Look Company,Look Up Revenue Line,Type,Source,Currency,...,FY-2029,FY-2030,FY-2031,FY-2032,FY-2033,FY-2034,FY-2035,FY-2036,FY-2037,FY-2038
0,New Drug,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra,Total,Actemra 1,Franc,...,976.607772,926.585167,982.631717,1025.784229,1066.754638,1060.295241,1243.330655,NaN,NaN,NaN
1,New Drug,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra - US,US,Actemra 2,Franc,...,319.258065,307.675764,334.707295,342.813380,362.472533,358.033808,362.957805,NaN,NaN,NaN


In [256]:
df = pd.read_excel('../data/raw data/raw data.xlsx', sheet_name = 'raw data')
df

,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Manufacturer,Look Company,Look Up Revenue Line,Type,Source,Currency,...,FY-2029,FY-2030,FY-2031,FY-2032,FY-2033,FY-2034,FY-2035,FY-2036,FY-2037,FY-2038
0,Actemra,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra,Total,Actemra 1,Franc,...,976.607772,926.585167,982.631717,1025.784229,1066.754638,1060.295241,1243.330655,NaN,NaN,NaN
1,Actemra,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra - US,US,Actemra 2,Franc,...,319.258065,307.675764,334.707295,342.813380,362.472533,358.033808,362.957805,NaN,NaN,NaN
2,Adempas,Riociguat,4.631524e+08,Bayer,Bayer,BAYGN_DE,Adempas (Riociguat),P1Total,Adempas 1,Euro,...,325.918579,331.983783,298.118641,126.244901,104.094874,86.948027,38.497813,NaN,NaN,NaN
3,Adempas,Riociguat,4.631524e+08,Bayer,Merck,MRK,International - Adempas,P2ExUS,Adempas 2,USD,...,222.045910,180.503438,118.701584,120.000000,119.732473,38.453391,26.917374,NaN,NaN,NaN
4,Adempas,Riociguat,4.631524e+08,Bayer,Merck,MRK,US - Adempas,P2US,Adempas 3,USD,...,240.144951,227.587172,256.503342,250.000000,249.940613,71.656430,50.159501,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Xtandi,Enzalutamide,1.968568e+09,Medivation,Pfizer/Astellas,Reports,Xtandi Global (Astellas Report),P2Total,Xtandi 4,Yen,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
214,Xtandi,Enzalutamide,1.968568e+09,Medivation,Pfizer/Astellas,Reports,Xtandi - US (Astellas Report),P2US,Xtandi 5,Yen,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
215,Xtandi,Enzalutamide,1.968568e+09,Medivation,Pfizer/Astellas,Reports,Xtandi - US (Astellas Report),P2US,Xtandi 6,USD,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
216,Yervoy,Ipilimumab,3.659614e+08,Bristol-Myers Squibb,BMS,BMY,Revenue - Yervoy,Total,Yervoy 1,USD,...,1250.745722,1046.866029,661.074817,563.407057,452.923421,768.705039,750.194071,NaN,NaN,NaN


,Proper Name,Generic Name,Medicare Spend,Original Manufacturer,Manufacturer,Look Company,Look Up Revenue Line,Type,Source,Currency,...,FY-2029,FY-2030,FY-2031,FY-2032,FY-2033,FY-2034,FY-2035,FY-2036,FY-2037,FY-2038
0,New Drug,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra,Total,Actemra 1,Franc,...,976.607772,926.585167,982.631717,1025.784229,1066.754638,1060.295241,1243.330655,NaN,NaN,NaN
1,New Drug,Tocilizumab,2.821445e+08,Genentech,Roche,ROG_CH,Actemra - US,US,Actemra 2,Franc,...,319.258065,307.675764,334.707295,342.813380,362.472533,358.033808,362.957805,NaN,NaN,NaN


0